# Data Science Application 2:
Exploratory Data Analysis and Machine Learning for Bias Diagnosis and Correction of a Groundwater Model

The following code snippets demonstrate a typical workflow that analyze errors of groundwater flow model simulation results, detect systematic error (bias), and perform bias correction using
machine learning. While the sample dataset used here comes from groundwater flow models, the workflow can be applied to other hydrologic models that generate spatio-temporal data.   

# 0. Install libraries
This section installs the `dataretrieval` and `hsclient` libraries from the GitHub repositories. `hsclient` will be used to download/create HydroShare resources, and `dataretrieval` will be used to obtain observational data from USGS NWIS database.

In [4]:
# # install the dataretrieval package, a Python alternative to USGS-R's dataRetrieval package
# !pip install  dataretrieval

# # # install the HydroShare RDF Python Client (dataretrieval) package
# !pip install hsclient
import matplotlib.colors as mcolors
import pyproj
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import dataretrieval.nwis as nwis
from hsclient import HydroShare
import statsmodels.api as sm
from sklearn.feature_selection import mutual_info_regression
from scipy.linalg import hankel
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.model_selection import train_test_split
import warnings

warnings.filterwarnings('ignore')


# 1. Data Retrieval
To get started, let's retrieve sample datasets from HydroShare and observational data from USGS NWIS.

## 1.1. Download a sample dataset from a HydroShare resource


The Python package `dataretrieval` allows you to download an entire HydroShare resource as a zipped file that uses the BagIt packaging standard. You can identify the resource you want to download using its HydroShare identifier. When you call the `download()` function on the resource, you can pass a path where you want to save the zipped file. Leaving the path blank downloads the files to the directory. For more details on how to use hsclient, refer to https://github.com/hydroshare/hsclient/ and the reference below.

For demonstration purpose, we will use a sample dataset containing the groundwater levels simulated by a groundwater flow model at locations where level observations are available from monitoring wells. The specifics are described in the readme.txt in the download and Xu et al. (2015).

**References**:  
Horsburgh, J. S., S. S. Black (2021). HydroShare Python Client Library (hsclient) Usage Examples, HydroShare, http://www.hydroshare.org/resource/7561aa12fd824ebb8edbee05af19b910

Xu, T., Valocchi, A. J., Choi, J., & Amir, E. (2014). Use of machine learning methods to reduce predictive error of groundwater models. Groundwater, 52(3), 448-460.

In [17]:
# Download file from hydro share
# Authentication is needed to read from/write to HydroShare resources

hs = HydroShare()
hs.sign_in()
uuid_repository='7152c722a45b407084954976a8b23aac' # the repository uuid you want to download

res = hs.resource(uuid_repository)
out_path= 'D:/DWSA/' # the folder path where you want to keep the download file
res.download(out_path)

Username:  shikita
Password for shikita:  ········


'D:/DWSA/7152c722a45b407084954976a8b23aac.zip'

## 1.2. Retrieve Observational Data from USGS NWIS
USGS National Water Information System (NWIS) is a database containing water resources data collected at over 1 million sites throughout CONUS. The data can be accessed through the NWIS webpage (https://waterdata.usgs.gov/nwis) or via R package `dataRetrieval` (https://cran.r-project.org/web/packages/dataRetrieval/vignettes/dataRetrieval.html) or  Python counterpart `dataretrieval`. Here, we use the Python package to retrieve groundwater level observations at selected wells. For more information check:
https://github.com/USGS-python/dataretrieval.




In [ ]:
# specify the USGS site code for which we want data.
# Simplest case: one groundwater observation well
site='410618098113401' # site number (water_id)
start_time='2007-10-01' # specify the desired start time from when data will be retrieved
end_time='2022-08-23'# specify ending time

# the get_record function retrieves data, option are: instantaneous values (iv), daily value (dv), statistics (stat),site info (site),discharge peaks (peaks),discharge measurements (measurements)
df_daily_value = nwis.get_record(sites=site, service='dv', start=start_time, end=end_time) # example of get daily water level value in time range (star, end)
# site info contains site characteritics such as land surface elevation and datum (for groundwater levels)
df_site_info = nwis.get_record(sites=site, service='site')

Data can also be downloaded in a batch mode for multiple side numbers. In the example below, we use a subset of the wells included in the `water_level_example.csv` file contained in the downloaded zip file to create a list of wells for data retrieval from NWIS. Only a subset is used so that the runtime is kept short. In a more general case, one could also specify a study area to obtain information of si within that area.

In [7]:
# This is an example of how to get the 'depth to water' of a list of wells.

file_path='D:/DWSA/water_level_example.xlsx'  # specify directory of this file
df=pd.read_excel(file_path)
# use the following commands when using Google Colab (may need to be tweaked for other online platforms)
# uploaded = files.upload()
# df = pd.read_csv(io.BytesIO(uploaded[file_path]))


In [8]:
df.iloc[0:4,:]

,well_no,x_coordinates,y_coordinates,date,MODFLOW,residual,weights,date_formated,well_id,observation
0,1,828290,266030,5128,1671.0,-2.169,0.1960,2004-09-14,474047117280601,1668.831
1,2,829600,258060,5072,1696.6,0.578,0.1886,2004-07-20,473928117275001,1697.178
2,2,829600,258060,5102,1695.0,-0.200,0.1922,2004-08-19,473928117275001,1694.800
3,2,829600,258060,5129,1694.5,1.051,0.1922,2004-09-15,473928117275001,1695.551


In [9]:
# get the groundwater level of  insitu USGS water level of well and date in df
demo_dataset = df.iloc[0:4,:] # use first 5 wells for demo. use the colume "well_id" to query NWIS, which are well IDs following USGS naming convention

# 1: for continuous measurements use option 'dv' to get daily mean or 'iv' for instantaneous value to get groundwater level, for example:
# df_demo = [nwis.get_record(sites=str(demo_dataset['well_id'][st]), service='dv', start=demo_dataset['date_formated'][st], end=demo_dataset['date_formated'][st]) for st in range (len(demo_dataset['well_id']))] # example of get daily water level for continuous measurements

# 2: for historic and discontinuous measurements, call 'site' and get the 'well_depth_va' to get depth to water and land surface altitude
# obtaining data of 'depth to groundwater table' in ft.
df_demo = [nwis.get_record(sites=str(demo_dataset['well_id'][st]), service='site', start=demo_dataset['date_formated'][st], end=demo_dataset['date_formated'][st])['well_depth_va'].item() for st in range (len(demo_dataset['well_id']))]

# get land surface altitude of the site. Detailed parameter description can be found : http://usgs-r.github.io/dataRetrieval/reference/readNWISsite.html
surface_altitude_demo = [nwis.get_record(sites=str(demo_dataset['well_id'][st]), service='site')['alt_va'].tolist() for st in range (len(demo_dataset['well_id']))]

# 'groundwater level' = 'surface altitude' - 'depth to water table'
water_level_observations_demo = np.array(surface_altitude_demo).flatten()-np.array(df_demo).flatten()

# add insitu observation to dataframe
demo_dataset['isnitu_water_level'] = water_level_observations_demo

## 1.3 Projection

#### This section is about projecting the model output to synchronize with USGS data, which is usually in latitude and longitude.

In [45]:
# This is an example of how to import the shp of model
shp_file_path='D:/DWSA/gis-colab/gis/1701_wb_shp.shp'#1701_area_shp#1701_wb_shp#active_cell_layer_3#svrp_model_boundary
# The 
gdf = gpd.GeoDataFrame(demo_dataset, geometry=gpd.points_from_xy(demo_dataset.x_coordinates, demo_dataset.y_coordinates))

# read the shapefile
df_shp = gpd.read_file(shp_file_path)
geo_shp = df_shp['geometry']

# Convert the polygon to a WKT string
geo_gdf = gpd.GeoDataFrame(geometry=geo_shp)

# Define the geographic coordinate system (replace XXXX with your desired GCS)
crs_latlon = pyproj.CRS.from_epsg(XXXX)

# Convert the polygon coordinates to geographic coordinates
geo_gdf2 = geo_gdf.to_crs(crs_latlon)

# 2. Error Diagnostics

## 2.1. Compute Residual
Residual is defined as observation minus model simulation results at the time and location of the observation.

In [ ]:
# y_mod : the data computed hydrology model
# y_insitu: the data from insitu observation
def computer_residual(y_mod, y_insitu):
    res = y_insitu - y_mod
    return(res)

y_mod = df['MODFLOW']
y_insitu = df['observation']
y_res = y_insitu - y_mod

## 2.2. Performance metrics
Here we include four commonly used performance metrics as defined below:
1. PBIAS: Percentage Bias (Gupta et al., 1999)
2. RMSE: Root Mean Sqaure Error;
3. NSE: Nash–Sutcliffe Efficiency;
4. KGE: Sutcliffe and Kling-Gupta Efficiency (Gupta et al., 2009)

NSE and KGE are best suited for time series data, for example, streamflow observations at one gaging station, and monthly groundwater level at a monitoring well

**References**

Gupta, H.V.; Sorooshian, S.; Yapo, P.O. Status of Automatic Calibration for Hydrologic Models: Comparison with Multilevel Expert Calibration. Journal of Hydrologic Engineering 1999, 4, 135–143, doi:10.1061/(ASCE)1084-0699(1999)4:2(135).

Gupta, H.V., Kling, H., Yilmaz, K.K. and Martinez, G.F., 2009. Decomposition of the mean squared error and NSE performance criteria: Implications for improving hydrological modelling. Journal of hydrology, 377(1-2), pp.80-91.



In [ ]:
def PBIAS(y_insitu, y_mod):
    a=np.sum(y_insitu-y_mod)
    b=np.sum(y_insitu)
    return (a/b)


def RMSE(y_insitu, y_mod):
    return(np.sqrt(np.mean((y_insitu-y_mod)**2)))


def NSE(y_insitu, y_mod):
    y_mean=y_insitu.mean()
    rss=np.sum((y_insitu-y_mod)**2)
    tss=np.sum((y_insitu-y_mean)**2)
    return (1-rss/tss)


def KGE(y_insitu, y_mod):
    variability_error=(np.std(y_insitu)/np.std(y_mod) -1)**2
    bia_error=(np.mean(y_insitu)/np.mean(y_mod)-1)**2
    linear_corre=(np.corrcoef(y_insitu,y_mod)[1,0] - 1)**2
    return(1-np.sqrt(variability_error+bia_error+linear_corre))


# Here we calculate NSE and KGE for individual sites having 10 or more
# observations, and then take the average across sites
def kge_nse(df, a,b):
# df is dataset, a,b are variables' name (in string) in df that you want to calculate KGE and NSE
    well_name=df['well_no']
    group_by=[df[well_name==i] for i in set(well_name)] # group  dataset by each site
    kge_per_well=[KGE(k[a].to_numpy(), k[b].to_numpy()) if len(k)>=10 else np.nan for k in group_by] # calculate KGE for sites have at least 10 observations
    nse_per_well=np.array([NSE(k[a].to_numpy(), k[b].to_numpy()) if len(k)>=10 else np.nan  for k in group_by]) # calculate NSE for have at least 10 observations
    return(kge_per_well,nse_per_well)

Below, we calculate the above metrics using the sample data or your own data. For the sample data, we found an overall small bias but substantial RMSE. The NSE and KGE scores show a wide range of variability across observation wells, with some wells having very low NSE and KGE scores, suggesting gap between simulation results and observations.

In [ ]:
well_name = df['well_no']

kge_per_well,nse_per_well = kge_nse(df, 'observation','MODFLOW')
nse_rm_inf_nan = nse_per_well[np.isfinite(nse_per_well)] # remove Inf value

print('pbias', PBIAS(df['observation'],df['MODFLOW']))
print('rmse', RMSE(y_insitu,y_mod))
print('nse', np.nanmedian(nse_rm_inf_nan))
print('kge', np.nanmedian(kge_per_well))

## 2.3. Residual plots
Various types of residual plots can be used to describe the statistical attributes of errors in more detail than the performance metrics above. These residual plots can reveal "structures" in the model residual, such as spatial and temporal correlation.

Scatter plot shows residual vs. observations.

In [ ]:
date = df['date_formated']
fig,axes=plt.subplots(nrows=1, ncols=1, sharex='all',figsize=(5,5),dpi=300)
plt.scatter(df['observation'],df['residual'], marker='*',c='red' ,s=1)
plt.xlabel('Observation (ft)')
plt.ylabel('Residual (ft)')
plt.title('Scatter plot of residual vs observation')
plt.show()

Time series plots at an individual site can be used to examine how well the model simulation replicates the observed dynamics at different locations of the study area. In the example below, we plot the observed and simulated groundwater level at two wells at which the model is behaving very well with the highest KGEs. It can be seen that the model overall captures the multi-year trend and seasonal fluctuation of water level, however seems to underestimate the peaks in summer and late fall/winter.

In [ ]:
"'1: Let's select 2 wells with highest KGE value'"

# combine NSE and KGE to dataframe for easy observation
nse_kge_dataframe=pd.DataFrame(list(zip(kge_per_well, nse_per_well,set(well_name))),  columns=['NSE','KGE', 'well_no'])
nse_kge_sort=nse_kge_dataframe.sort_values(by='NSE', ascending=False) # rank discending base on NSE value. Can also replace NSE by KGE.

# select top 2 wells
well_1_no=nse_kge_sort.iloc[1]['well_no'].astype('int')
well_2_no=nse_kge_sort.iloc[2]['well_no'].astype('int')

well_2=df[df['well_no']==well_2_no]
well_1=df[df['well_no']==well_1_no]

x_index_1=pd.to_datetime(well_1['date_formated'])
x_index_2=pd.to_datetime(well_2['date_formated'])

"'2: Let's plot selected wells time series'"
import matplotlib.dates as mdate

fig,axes=plt.subplots(nrows=1, ncols=2, figsize=(10,5),dpi=300)
axes[0].plot(x_index_1, well_1['observation'],'o-',label='Well No. '+str(well_1_no)+' observation')
axes[0].plot(x_index_1, well_1['MODFLOW'],'*-',label='Well No. '+str(well_1_no)+' MODFLOW')
axes[1].plot(x_index_2, well_2['observation'],'o-',label='Well No. '+str(well_2_no)+' observation')
axes[1].plot(x_index_2, well_2['MODFLOW'],'*-',label='Well No. '+str(well_2_no)+' MODFLOW')

axes[0].set_xlabel('Time')
axes[0].set_ylabel('Water level (ft)')
# axes.set_title('Time series plot')
axes[0].tick_params(direction='in')
axes[0].set_xticklabels(x_index_1,rotation=90)
monthFmt = mdate.DateFormatter('%Y-%m')
axes[0].xaxis.set_major_formatter(monthFmt)
axes[0].legend()

axes[1].set_xlabel('Time')
axes[1].set_ylabel('Water level (ft)')
# axes.set_title('Time series plot')
axes[1].tick_params(direction='in')
axes[1].set_xticklabels(x_index_1,rotation=90)
monthFmt = mdate.DateFormatter('%Y-%m')
axes[1].xaxis.set_major_formatter(monthFmt)
axes[1].legend()

plt.show()


Plotting time-averaged residuals in a map shows spatial trends of residuals and can reveal locations where the model is systematically over- or under- predicting. In the example below, large residuals mostly occur near the model boundary and near the Spokane River.

In [ ]:
df_time_average=df.groupby(df['well_no']).mean() # get the temporal average value of each site

x_time_average=df_time_average['x_coordinates']
y_time_average=df_time_average['y_coordinates']
res_time_average=df_time_average['residual']
fig,axes=plt.subplots(nrows=1, ncols=1, sharex='all',figsize=(5,5),dpi=300)

offset = mcolors.TwoSlopeNorm(vmin=res_time_average.min(),
                                  vcenter=0, vmax=res_time_average.max())

plt.scatter(x_time_average, y_time_average, c=res_time_average, norm=offset,cmap='RdBu')
plt.title("Point observations")
plt.xlabel("x (mi.)")
plt.ylabel("y (mi.)")
cbar= plt.colorbar()
cbar.set_label('Residual (ft)', labelpad=+1)
plt.show()

Finally, the Q-Q (quantile-quantile) plot can be used to test whether the error is white noise, i.e., i.i.d. normal with a zero mean. A Q-Q plot graphically compares the distribution of the residual with a reference distribution (a normal distribution is used here). If the residual follows a normal distribution, the plots in the Q-Q plot will approximately lie on the identity line (y=x).

In [ ]:


def qqplot(x):
    fig, ax = plt.subplots(ncols=1,nrows=1, figsize=(5,5),dpi=300)

    sm.qqplot(x, fit=True, line ='45', ax=ax)
    ax.grid()
    ax.set_title('Q-Q plot of the residual',fontsize=15)
    ax.xaxis.get_label().set_fontsize(12)
    ax.yaxis.get_label().set_fontsize(12)
    ax.get_lines()[1].set_color("red")
    ax.get_lines()[1].set_linewidth("2")
    plt.show()
    return(fig, plt.show())



Using the sample dataset, the Q-Q plot overall appears steeper than the line y=x, indicating the residual distribution is more dispersed than a normal distribution. In addition, the Q-Q plot is "S" shaped, suggesting the residual distribution is skewed and heavy tailed.

In [ ]:
qqplot(y_res)

## 2.4. Correlation analyses
Next, we will calculate and visualize some statistics regarding the correlation structures of the residual. Specifically, we examine autocorrelation and spatial correlation as below.


ACF, or autocorrelation function, characterize the correlation of a time series with a delayed copy of itself as a function of delay. ACF is often used to find periodic (e.g., seasonal, diurnal) patterns.

In [ ]:
def ACF(x, lag): # lag should <= length of x
    if lag <= len(x):
        n=len(x)
        return (np.corrcoef(x[:n-lag], x[lag:])[1,0])
    else:
        return(print('lag should <= length of x, in this case it is '+ str(len(x))))

In [ ]:
def plot_acf(well_acf_no,df,variable_name, lag): #well_acf_no:  well_no you want to plot, lag should <= length of x
    well_acf=df[df['well_no']==well_acf_no]
    ACF(well_acf[variable_name],lag)
    acf_plot=[ACF(well_acf[variable_name],i) for i in range (lag)]
    "'selected well acf plot'"
    plt.plot(list(range(lag)), acf_plot, '*-')
    plt.xlabel('Lags (Months)')
    plt.ylabel('Correlation coefficient')
    plt.title('ACF of well ' +str(well_acf_no) )
    return(fig)

In [ ]:
fig, ax=plt.subplots(ncols=1,nrows=1,figsize=(5,5), dpi=300)
plot_acf(92,df,'residual',40)

plt.show()

We use variogram to analyze the spatial correlation structure of residuals. For more information about variogram, please refer to [here](https://www.sciencedirect.com/topics/earth-and-planetary-sciences/variogram#:~:text=A%20variogram%20is%20defined%20as,Comprehensive%20Geographic%20Information%20Systems%2C%202018).



In [ ]:
def dist(x, y, res): # here lon, lat, res, should be in numpy array or list
    diff_y=[(k-v)**2/2 for k in res for v in res]
    diff_dis=[ ((x[i] -x[j] )**2 + (y[i]-y[j])**2)**0.5 for i in range(len(x)) for j in range(len(y))] # Euclidean distance
    return (diff_y, diff_dis)


import statistics


def kg_plot(diff_y, diff_dis,n_bin,res_time_average):
    x_plot=np.linspace(start=min(diff_dis), stop=max(diff_dis), num=n_bin)
    d={'dis': diff_dis, 'res': diff_y}
    new_df=pd.DataFrame(data=d)
    plt_y=new_df['res'].groupby(pd.cut(new_df["dis"], x_plot)).mean()
    fig, ax=plt.subplots(ncols=1,nrows=1,figsize=(5,5), dpi=300)
    plt.plot(x_plot[:-1]/5280,plt_y,'s-',label='Residual semi-variogram')
    plt.plot(x_plot[:-1]/5280,[statistics.variance(res_time_average) for k in range (len(x_plot[:-1]))], '--', label='Residual variance')
    plt.legend()
    plt.xlabel('Distance (mi.)')
    plt.ylabel(r'$\gamma (ft^2)$')
    plt.title('Variogram')
    return (fig, plt.show())

We plot the semi-variogram of the time average of residuals at all wells. In the example below, the semi-variogram increases with distance, suggesting spatial correlation exists. This is consistent with the spatial patterns observed in the spatial plot of time average of residuals.

In [ ]:
diff_y, diff_dis = dist(x_time_average.to_numpy(), y_time_average.to_numpy(), res_time_average.to_numpy())
n_bin = 30
kg_plot(diff_y, diff_dis,n_bin,res_time_average)

# 3. Bias Correction using Machine Learning

The residual analyses and plots performed on the sample dataset revealed that the groundwater model simulation results contain systematic error (or bias) even though the model has been calibrated to overall good accuracy. Bias may come from inevitable simplifications when we build numerical models of the complicated reality, model parameters not fully representing the heterogeneity, or errors in the input data. These sources of biases are difficult to address using a physically based groundwater model. In this case, machine learning can be used to correct these biases.

In this demo, we will build machine learning models to characterize the function between residual (y) and explanatory variables (x). We can then use the machine learning predicted residual to bias correct simulation results of groundwater models.

## 3.1. Feature engineering
Conventional machine learning algorithms would often benefit from feature engineering guided by expert knowledge, particularly when a large number of explanatory variables are present and are potentially related to the target variable. Feature engineering is the process of selecting input variables, sometimes involving transforming the input variables. Here we use mutual information and multivariate singular spectrum analyses.

### Mutual Information

For two variables, mutual information is the score that describle in what level one variable can explain the other. This example use the `normalized_mutual_info_score` function in `scikit-learn` to calculate the normalized version of mutual information, i.e., in the range of (0,1), with 0 suggesting no relation between the two variables.

**Reference**

Learned-Miller, E.G., 2013. Entropy and mutual information. Department of Computer Science, University of Massachusetts, Amherst, p.4. access : https://people.cs.umass.edu/~elm/Teaching/Docs/mutInf.pdf

In [ ]:
# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.normalized_mutual_info_score.html

#kk=normalized_mutual_info_score(x, y)




### Multivariate Singular Spectrum Analysis (MSSA)

Here, we provide a brief, intuitive description of MSSA. More technical details can be found in the references below. Singular Spectrum Analysis (SSA) is an extension of the classical principal component analysis. Given a time series, SSA computes a lag-covariance matrix and its eigenvalues and eigenvectors (empirical orghogonal functions, or EOFs). It then decompose the time series into temporal principle components by projecting the time series onto each EOF. MSSA is an extension of SSA to multivariate time series by calculating the lag-covariance matrix across the variables.

**References**

Hassani, H. and Mahmoudvand, R., 2013. Multivariate singular spectrum analysis: A general view and new vector forecasting approach. International Journal of Energy and Statistics, 1(01), pp.55-83.

Singular-Spectrum Analysis: SSA notebook: https://www.kaggle.com/code/jdarcy/introducing-ssa-for-time-series-decomposition/notebook


In [ ]:


# normalise input variables
def normalization_df(df):
    normalized_df=(df-df.min())/(df.max()-df.min())
    return(normalized_df)


# embeding - form the trajectory matrix with lagged copies of the time series
def svd(trajmat):

    u, s, vh = np.linalg.svd(trajmat)
    d = np.linalg.matrix_rank(trajmat)
    X_elem = np.array( [s[i] * np.outer(u[:,i],vh[i,:]) for i in range(d)] )
    return(X_elem)


# diagonal averaging
def reconstruction(X_i):
    # Reverse for anti diagonal
    X_rev =np.flipud(X_i)
    # Full credit to Mark Tolonen at https://stackoverflow.com/a/6313414 for this one:
    return(np.array([X_rev.diagonal(i).mean() for i in range(-X_i.shape[0]+1, X_i.shape[1])]))


# Reconstruct time series and plot
def rcons_plot(noiseless_level,X_elem,d, df_plot): # d is rank of embedding mat

    n = min(noiseless_level,d) # In case of noiseless time series with d < 12.

    # Convert elementary matrices straight to a time series
    start=0
    fig,axes=plt.subplots(4,1, sharex='all',sharey='all',figsize=(10,10),dpi=300)
    axes = np.array(axes).flatten()
    for v in range(4):

        end=(v+1)*389
        for i in range(n):
            hhh=X_elem[i][:,start:end]
            F_i = reconstruction(hhh)
            x_plot=list(range(len(F_i)))
            axes[v].plot(x_plot, F_i, lw=2)
        start=end-1
        # x_plot_2=list(range(len(df_plot.iloc[:,v])))
        axes[v].plot(list(range(len(df_plot.iloc[:,v]))), df_plot.iloc[:,v], alpha=1, lw=1)


        axes[v].set_ylabel(r"$\tilde{F}_i(t)$")
        legend = [r"$\tilde{F}_{%s}$" %i for i in range(n)] + ["$F$"]
        axes[v].set_title("The First " + str(n) + " Components of " + df_plot.iloc[:,v].name)
        plt.legend(legend, loc=(1.05,0.1));
        if v>2:
            axes[v].set_xlabel("$t$")
    return()



### Example
To illustrate the usage of MSSA, we use a second sample dataset from Xu et al. (2015). The dataset includes the residual of baseflow (referring to groundwater discharge to rivers) simulated by a groundwater flow model when compared to baseflow estimations determined from hydrograph separation. The sample dataset contains several variables that are potentially related to the baseflow residual, including:

*   River segment id
*   Precipitation in the vicinity of river segment
*   Potential evapotranspiration (PET) in the vicinity of river segment
*   Estimated irrigation amount in the vicinity of river segment
*   Model computed groundwater levelin the vicinity of river segment
*   Model computed baseflow

Here, we will use MSSA to determine the degree of relevancy of these variables to baseflow residual.

**Reference**

Xu, T., & Valocchi, A. J. (2015). Data-driven methods to improve baseflow prediction of a regional groundwater model. Computers & Geosciences, 85, 124-136.

In [ ]:
# If running Jupyter notebook locally
file_path_2 ='discharge_example.csv' # included in the downloaded HydroShare resource
#df_2 = pd.read_csv(file_path_2)

# If using Google Colab (may need to be tweaked for other online platforms)
uploaded = files.upload()
df_2 = pd.read_csv(io.BytesIO(uploaded[file_path_2]))

In [ ]:
"'Exmaple of mutual information score'"
# import sys
# print(sys.setrecursionlimit(2000))
df_2.columns
mi_input=df_2.columns[:-1]
mi_val=mutual_info_regression(df_2[mi_input],df_2['res'])
fig, ax=plt.subplots(ncols=1,nrows=1,figsize=(5,5), dpi=300)
plt.bar(list(range( len(mi_val))),mi_val,label='Mutual information score')
plt.xticks(list(range( len(mi_val))),mi_input)
plt.legend()
plt.xlabel('Variables')
plt.ylabel('Mutual information score')
plt.title('Mutual information')



In [ ]:
"'Exmaple of MSSA'"
# first, normalize data
df_2_norm = normalization_df(df_2)
# row of trajectory (embedded) matrix
window = 100
N = len(df_2_norm['irr'])
# length.
K = N - window+ 1 # The number of columns in the trajectory matrix.
# embedding variable to mat
trajmat_1 = hankel(df_2_norm['irr'][0:window], r=df_2_norm['irr'][window:])
trajmat_2 = hankel(df_2_norm['Qcomp'][0:window], r=df_2_norm['Qcomp'][window:])
trajmat_3 = hankel(df_2_norm['ppt'][0:window], r=df_2_norm['ppt'][window:])
trajmat_4 = hankel(df_2_norm['evt'][0:window], r=df_2_norm['evt'][window:])
trajmat = np.concatenate((trajmat_2,trajmat_1, trajmat_4,trajmat_3), axis=1)
# decompose
d = np.linalg.matrix_rank(trajmat)
X_elem = svd(trajmat)

X_name = ['irr','Qcomp','ppt','evt']
df_plot = df_2_norm[X_name]
rcons_plot(7,X_elem,d, df_plot)

# 3.2. Data preprocessing

Before using machine learning for bias correction, it's a good idea pre-process data for numerical stability. Two commonly used linear scaling practices are:
(1) Standardization, which scales data to have a mean of 0 and standard deviation of 1.
(2) Normalization, which scales data into a range of `[0,1]`.
In general, normalization works better if the dataset has outliers. Note that in cases where data distribution is very skewed and/or tailed, it may be desirable to perform nonlinear transformation to make data more "normally" distributed and easier for machine learning modeling.

In [ ]:


# Standardization
def Standardization_df(df):
    std_df=(df-df.mean())/df.std()
    return(std_df)


# Normalization
def normalization_df(df):
    normalized_df=(df-df.min())/(df.max()-df.min())
    return(normalized_df)

def scale_back(df_norm, df):
    scale_back_df=df_norm*(df.max()-df.min()) +df.min()
    return(scale_back_df)

In [ ]:
X_name = ['x_coordinates','y_coordinates','date','MODFLOW']
df_y = normalization_df(df['residual'])
# df_y = df['residual']
# in this example we choose to normalize x (explanatory) variables
df_X = normalization_df(df[X_name])

Next, we will randomly split the dataset into training and test portions. We will use the training set to train the machine learning models and test to assess the generalization performance of the machine learning model.

In [ ]:
# for reproducibility, we set a random seed here. You can use arbitrary value
random_seed = 117
# fraction of test data
split_fraction = 0.2
# Splitting into training, validation, and test subsets
X_tr,  X_te, y_tr, y_te, df_tr,df_te=train_test_split(df_X,df_y,df,test_size=split_fraction, random_state=random_seed)
# check the size
X_tr.shape, y_tr.shape, X_te.shape

# 3.3. Gaussian Process Regression

Gaussian process regression (GPR) is a Bayesian kernel regression method that uses a Gaussian Process (GP) to describe the distribution of a variable and the Bayes' theorem to infer the posterior distribution conditioned on data. An advantage of GPR is that it is inherently Bayesian and thus can provide uncertainty estimates associated with prediction. For technical details, plese refer to Williams and Rasmussen (2006). Here, we use the `sklearn.gaussian_process` package.




**Reference**

Williams, C. K., & Rasmussen, C. E. (2006). Gaussian processes for machine learning (Vol. 2, No. 3, p. 4). Cambridge, MA: MIT press.


A GP is fully specified by a mean function and a covariance function (also known as kernel). Before seeing any of the data, we have the *a priori* that the residual should have a zero mean and a squared exponential kernel, which is commonly used as the first choice in various applications. The kernel has two hyperparameters, namely *range* and *sill*, that control the correlation scale (similar to the range in the variogram) and signal variance, respectively. With training data, the two hyperparameters can be determined by maximizing likelihood. Using the Bayes' theorem, GPR let the observational data to "sculpt" the prior into posterior, which can be evaluated at unseen points to generate prediction.

In [ ]:
from sklearn.gaussian_process.kernels import RBF
# use other kernel as needed.
# useful link of avaliable kernel and instruction:
# https://scikit-learn.org/stable/modules/gaussian_process.html

def gpr_pred(X_train, y_train,X_te, Kernel):
  # hyperparameters are determined during training
    gpr = GaussianProcessRegressor(kernel=Kernel,
        random_state=random_seed).fit(X_train, y_train)
    # making predictions
    y_pred = gpr.predict(X_te)
    print('Testing score:')
    print(gpr.score(X_te, y_pred))
    return(y_pred)

Running the defined `gpr` regressor will output GPR predicted residual at test points. We will evaluate the accuracy later.

The length_scale determines the length of the 'wiggles' in your function. In general, you won't be able to extrapolate more than this number away from your data. The smaller the number the complex the model, thus more likely to overfit.

The parameter in front of kernel is  variance σ2; determines the average distance of your function away from its mean. it is actually a scale factor.

The parameter add to kernel is offset; the mean of distribution.

In [ ]:
kernel = 1* RBF(length_scale=0.01, length_scale_bounds=(1e-05, 1))+0.39
yte_pred_gs_norm = gpr_pred(X_tr, y_tr, X_te, kernel)
yte_pred_gs = scale_back(yte_pred_gs_norm, df['residual'])

## 3.4. Random Forest

The Random Forests are an ensemble learning method that use the "bagging" technique to combine many decision trees in order to improve the statistical stability of decision trees and reduce overfitting. More specifically, random forest trains *N* trees, with each tree fitted to a bootstrap sample (i.e. sample with replacement) of the original training data.

### OOB error
Each bootstrap sample leaves out about one-third of the training data, which is called the out-of-bag (OOB) data. The OOB error is the average error of a tree evaluated on the OOB data. Since for each tree, the OOB data is unseen during training, OOB error provides a convenient way to estimate generalization error. OOB error can also be used to assess the importance of each input variables by permuting each variable in turn and track the resulting increase in OOB error.

### Hyperparameters
`n_estimators`: Number of trees in the forest.

`max_features`: The maximum number of variables used at each split for each tree. This item is used to avoid overfitting.

`min_samples_split`: The minimum number of samples required to split and grow new branches at an internal node. The smaller the number, the higher the chance model tends to overfit.

`min_samples_leaf`: The minimum number of samples required to be at a leaf node. The smaller the number, the higher the chance model tends to overfit.

We will utilize the `GridSearchCV` function in the `sklearn.model_selection` package to select hyperparameters.

**Reference**

https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html



In [ ]:
# Grid search to tune hyperparameters using oob (no need to reserve a validation set). Number of trees, random subset per split, leaf size
# Use best hyperparameters to train and test

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
# X train is input features
# y _train is target

# self define score function for Gridsearch, using random forest oob error as score/ criteria
def oob_scorer(estimator, X, y):
    return estimator.oob_score_

# set up the hyperparameter grids. For each hyperparameter combination, train a
# and evaluate OOB error
param_grid = {
    'n_estimators':np.arange(100,500,100),
    'max_features' : [2,3],
    'min_samples_split' : np.arange(2,32,4),
    'min_samples_leaf' : np.arange(2,32,4),
}

def Grid_search(X_train, y_train,param_grid):
# random forest regression
    rfr = RandomForestRegressor(n_estimators = 500, n_jobs = -1,random_state=random_seed,oob_score=True)
    #set up grid search
    model_rf = GridSearchCV(estimator=rfr,
                      param_grid=param_grid,
                      scoring=oob_scorer,
                      )
    # fit and train
    model_rf.fit(X_train, y_train)

    #print best parameters
    model_rf.best_params_

    #print the best train score (oob error)
    model_rf.score(X_train, y_train)

    return(model_rf.best_params_)
    # predict test data
def RFM(X_tr, y_tr,X_te, best_par):
    rfr_2=  RandomForestRegressor(n_estimators =best_par['n_estimators'],max_features=best_par['max_features'], min_samples_leaf=best_par['max_features'],

                                  min_samples_split=best_par['min_samples_split'],n_jobs = -1,random_state=random_seed,oob_score=True) # set RF model with best parameter
    rfr_2.fit(X_tr, y_tr) # fit RF with Train data set
    yte_pred=rfr_2.predict(X_te) # RF test
    return(yte_pred)

In [ ]:
# determine best hyperparameters
best_par = Grid_search(X_tr, y_tr,param_grid)
print(best_par)
# make predictions
yte_pred_rf_norm = RFM(X_tr, y_tr,X_te, best_par)
# scale back
yte_pred_rf = scale_back(yte_pred_rf_norm , df['residual'])

# 3.5. Artificial Neural Networks

# Multi-layer Perceptron (MLP)
MLP, also known as feedforward neural networks, are inspired by biological learning processes and built out of a densely interconnected set of units or neurons. A typical MLP network consists of an input layer, one or more hidden layers and an output layer. Information flows through the connections between units. Each unit computes a single output by passing the weighted sum of its inputs plus a bias term through a nonlinear activation function (e.g., sigmoid or rectifier).

In this example, we build a simple MLP model for bias correction. We need to tune the model hyperparameters, including number of hidden layers, number of hidden units in each layer, type of the activation function, and learning rate.

The learning rate determines the size of updates applied on the weights in each training iteration (i.e., epoch). A too low learning rate results in very long training process, while a too high learning rate may overshoot and have trouble finding the optimum.

Number of hidden layers and units determine the complexity of the network; for this relatively simple example, we keep them small so the model won't overfit.

Use 'early_stopping' to avoid overfitting;
It will automatically set aside 10% of training data as validation and terminate training when validation score is not improving by at least 1e-4 (By default,  can be adjust by 'tol') for 10 (n_iter_no_change) consecutive epochs. Only effective when solver=’sgd’ or ‘adam’. quoted from: (https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html)


In [ ]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV
param_grid_MLP = {
    'hidden_layer_sizes':[(16,2),(8,1),(16,1)],
    'activation' : ['relu','sigmoid','tanh'],
    'solver': ['sgd', 'adam'],
    'learning_rate_init' :np.arange(0.00001,0.001,0.0001),
}


def Grid_search_MLP(X_train, y_train,param_grid):
# MLP
    regr = MLPRegressor(random_state=random_seed,early_stopping=True, max_iter=500)
    #set up grid search
    model_mlp = GridSearchCV(estimator=regr,param_grid=param_grid, cv=5)

    # fit and train
    model_mlp.fit(X_train, y_train)

    #print best parameters
    model_mlp.best_params_

    #print the best train score (oob error)
    model_mlp.score(X_train, y_train)

    return(model_mlp.best_params_)


def mlp_pre(X_tr, y_tr,X_te, best_par):
    regr_2 = MLPRegressor(random_state=random_seed, max_iter=500,
                          hidden_layer_sizes=best_par['hidden_layer_sizes'],
                          activation=best_par['activation'],
                          solver=best_par['solver'],
                          learning_rate_init=best_par['learning_rate_init'],
                          early_stopping=True)

    regr_2.fit(X_tr, y_tr) # fit RF with Train data set
    yte_pred=regr_2.predict(X_te) # RF test
    return(yte_pred)

In [ ]:
# determine best hyperparameters
best_par_mlp = Grid_search_MLP(X_tr, y_tr,param_grid_MLP)
print(best_par_mlp)
# make predictions
yte_pred_mlp_norm = mlp_pre(X_tr, y_tr,X_te, best_par_mlp)
# scale back
yte_pred_mlp = scale_back(yte_pred_mlp_norm , df['residual'])

# 4. Performance Evaluation
Now that we have built three machine learning models for bias correction, let's assess how well each of them performs. We will use the residual predicted by each model on the test data to update the groundwater model simulation results, calculate the remaining residual after updating, and then perform diagnostics similarly as in Section 3 to see the effectiveness of bias correction.

## 4.1. Performance metrics after bias correction
First, let's re-calculate the performance metrics used in 2.1, now after machine learning-based bias correction. For the sample dataset, the bias correction led to PBIAS closer to zero, lower RMSE, and higher NSE and KGE for the test dataset, all suggesting better match to observations.

In [ ]:
df_te['ml_GPR'] = df_te['MODFLOW'] + yte_pred_gs
df_te['ml_RF'] = df_te['MODFLOW'] + yte_pred_rf
df_te['ml_MLP'] = df_te['MODFLOW'] + yte_pred_mlp
for ml_index in ['ml_GPR','ml_RF','ml_MLP']:#,
    kge_per_well_te, nse_per_well_te = kge_nse(df_te, 'observation',ml_index)
    nse_rm_inf_nan_te = nse_per_well_te[np.isfinite(nse_per_well_te)] # remove Nan and Inf values if there are
    print('PBIAS of '+ ml_index +':', PBIAS(df_te['observation'],df_te[ml_index]))
    print('RMSE '+ ml_index +':', RMSE(df_te['observation'],df_te[ml_index]))
    print('NSE '+ ml_index +':', np.nanmedian(nse_rm_inf_nan_te))
    print('KGE '+ ml_index +':', np.nanmedian(kge_per_well_te))

## 4.2. Residual plots after bias correction

In [ ]:
fig,axes=plt.subplots(nrows=2, ncols=2, sharex='all',figsize=(10,10),dpi=500)
axes = axes.flatten()

model_result=['MODFLOW','ml_RF','ml_GPR','ml_MLP']
plot_legend=['Before BC','BC with RF','BC with GPR','BC with MLP']
for n_model in range (len(model_result)):
    axes[n_model].scatter(df_te['observation'],df_te['observation']-df_te[model_result[n_model]],
                        marker='*',c='red' ,s=1, label=plot_legend[n_model])
    axes[n_model].set_xlabel('Observation (ft)')
    axes[n_model].set_ylabel('Residual (ft)')
    axes[n_model].legend()
    axes[n_model].set_ylim([min(df_te['residual']), max(df_te['residual'])])
fig.tight_layout()
plt.show()

Next, let's plot out groundwater level time series at a few wells. The figure below compares the observed groundwater level, groundwater simulated (No BC), and after bias correction (BC).

In [ ]:
import matplotlib.dates as mdate
random_pick = [236,251,2,13]
print(random_pick)

fig,axes = plt.subplots(nrows=2, ncols=2,figsize=(10,10),dpi=300)
fig.autofmt_xdate(rotation=90)
for n in range (4):
# sort data based on its time
    well_plt = df_te[df_te['well_no']==random_pick[n]]
    well_plt['date_formated'] = pd.to_datetime(well_plt['date_formated'])
    well_sorted = well_plt.sort_values(by=['date_formated'])

    # plot
    axes = axes.flatten()
    axes[n].plot(well_sorted['date_formated'], well_sorted['observation'], '-o', c='g', label='Obs.')
    axes[n].plot(well_sorted['date_formated'], well_sorted['MODFLOW'], '-^', c='r', label='No BC')
    axes[n].plot(well_sorted['date_formated'], well_sorted['ml_RF'],'-*', c='b', label='BC w/ RF')
    axes[n].plot(well_sorted['date_formated'], well_sorted['ml_MLP'],'-*', c='orange', label='BC w/ MLP')
    axes[n].plot(well_sorted['date_formated'], well_sorted['ml_GPR'],'-*', c='grey', label='BC w/ GPR')
    axes[n].set_title( 'Well No. ' + str(random_pick[n]))
    axes[n].set_ylabel('(ft)')
    axes[n].set_xlabel('Time')
    monthFmt = mdate.DateFormatter('%Y-%m')
    axes[n].xaxis.set_major_formatter(monthFmt)
    axes[n].legend()

fig.tight_layout()
plt.show()


In [ ]:
df_time_average_te=df_te.groupby(df_te['well_no']).mean() # get the temporal average value of each site

x_co_time_average_te=df_time_average_te['x_coordinates']
y_co_time_average_te=df_time_average_te['y_coordinates']

fig,axes=plt.subplots(nrows=2, ncols=2, sharex='all',figsize=(10,10),dpi=300)
axes = axes.flatten()
plot_bc=['MODFLOW','ml_RF','ml_GPR','ml_MLP']
plot_title=['No BC','BC w/RF','BC w/GPR','BC w/MLP']
for n in range(len(plot_bc)):#,
    bc_index=plot_bc[n]
    offset = mcolors.TwoSlopeNorm(vmin=-30,
                                  vcenter=0, vmax=30)
    im1=axes[n].scatter(x_co_time_average_te, y_co_time_average_te, c=df_time_average_te['observation']-df_time_average_te[bc_index], norm=offset,cmap='RdBu')
    axes[n].set_title(plot_title[n])
    axes[n].set_xlabel("X (mi.)")
    axes[n].set_ylabel("y (mi.)")
    fig.colorbar(im1, ax=axes[n],label='Residual (ft)')
fig.tight_layout()
plt.show()

In [ ]:

plot_bc=['MODFLOW','ml_RF','ml_GPR','ml_MLP']
plot_title=['No BC','BC w/RF','BC w/GPR','BC w/MLP']
fig, axes=plt.subplots(ncols=2,nrows=2,sharex=True, sharey=True,figsize=(10,10), dpi=300)
axes = axes.flatten()
lag=20
well_plot_no_te=92
well_acf_te=df_te[df_te['well_no']==well_plot_no_te]
for n_model in range (len(plot_bc)):
    plot_res=well_acf_te['observation']-well_acf_te[plot_bc[n_model]]
    acf_plot=[ACF(plot_res,i) for i in range (lag)]
    axes[n_model].plot(list(range(lag)), acf_plot, '*-',label=plot_title[n_model])
    axes[n_model].set_xlabel('Lags (Months)')
    axes[n_model].set_ylabel('Correlation coefficient')
    axes[n_model].legend()

fig.tight_layout()
plt.show()



In [ ]:

plot_bc=['MODFLOW','ml_RF','ml_GPR','ml_MLP']
plot_title=['No BC','BC w/RF','BC w/GPR','BC w/MLP']
fig, axes=plt.subplots(ncols=2,nrows=2,sharex=True,figsize=(10,10), dpi=300)
axes = axes.flatten()
n_bin = 30

for n_model in range (len(plot_bc)):
    res_te=df_time_average_te['observation']-df_time_average_te[plot_bc[n_model]]
    diff_y_te, diff_dis_te = dist(df_time_average_te['x_coordinates'].to_numpy(), df_time_average_te['y_coordinates'].to_numpy(), res_te.to_numpy())
    x_plot_te=np.linspace(start=min(diff_dis_te), stop=max(diff_dis_te), num=n_bin)
    d_te={'dis': diff_dis_te, 'res': diff_y_te}
    new_df_te=pd.DataFrame(data=d_te)
    plt_y_te=new_df_te['res'].groupby(pd.cut(new_df_te["dis"], x_plot_te)).mean()
    axes[n_model].plot(x_plot_te[:-1]/5280,plt_y_te,'s-',label=plot_title[n_model]+' residual semi-variogram')
    axes[n_model].plot(x_plot_te[:-1]/5280,[statistics.variance(res_te) for k in range (len(x_plot_te[:-1]))], '--', label=plot_title[n_model]+' residual variance')
    axes[n_model].set_xlabel('Distance (mi.)')
    axes[n_model].set_ylabel(r'$\gamma (ft^2)$')
    axes[n_model].legend()
fig.tight_layout()
plt.show()


## 4.3. Write the bias corrected model outputs back to HydroShare

Finally, we will save the bias corrected results and upload to HydroShare. For more technical details, refer to the documentation of `hsclient` named `file_operation_ipyn`.


In [ ]:
"'Machine learning model to bias correction all samples'"
pred_gs_norm = gpr_pred(X_tr, y_tr, df_X, kernel)
pred_gs = scale_back(pred_gs_norm, df['residual'])

pred_rf_norm = RFM(X_tr, y_tr,df_X, best_par)
# scale back
pred_rf = scale_back(pred_rf_norm , df['residual'])


pred_mlp_norm = mlp_pre(X_tr, y_tr,df_X, best_par_mlp)
# scale back
pred_mlp = scale_back(pred_mlp_norm, df['residual'])


In [ ]:
"'Save machine learning result to dataset'"
df['BC w/RF']=pred_rf+df['MODFLOW']
df['BC w/GPR']=pred_gs+df['MODFLOW']
df['BC w/MLP']=pred_mlp+df['MODFLOW']

In [ ]:
"'Save new dataset to local'"
output_path='' # it is better to save it to Microsoft Excel Worksheet (.xlsx) formate. Since csv automatically do scientific notation for value has more than 12 digitial numbers.
df.to_excel(output_path,index=False)

In [ ]:
"'Upload to hydroshare'"
# Create the new, empty resource
hs = HydroShare()
hs.sign_in()
new_resource = hs.create()

# Get the HydroShare identifier for the new resource
resIdentifier = new_resource.resource_id
print('The HydroShare Identifier for your new resource is: ' + resIdentifier)

# Construct a hyperlink for the new resource
print('Your new resource is available at: ' +  new_resource.metadata.url)

# Upload one or more files to your resource
file_upload_path=''
new_resource.file_upload(file_upload_path)

# Print the names of the files in the resource
print('Updated file list after adding a file: ')
for file in new_resource.files(search_aggregations=True):
    print(file.path)